In [1]:
import csv
import ArgConfigParse
from dateutil import rrule
from datetime import datetime


In [2]:
config = ArgConfigParse.ConfigFile(['setup.ini'])

In [3]:
config.parse_config()

{'dates': {'start': '2021/08/18', 'end': '2022/06/17'}}

In [4]:
config.config_dict

{'dates': {'start': '2021/08/18', 'end': '2022/06/17'}}

In [20]:
with open('./vacation_days.txt', 'r') as vacation_file:
    vacation_txt = vacation_file.readlines()

vacation_dt = []
for i in vacation_txt:
    try:
        vacation_dt.append(datetime.strptime(i.strip(), '%Y/%m/%d'))
    except ValueError as e:
        pass


In [54]:
days = []
for dt in rrule.rrule(rrule.DAILY, dtstart=datetime.strptime(config.config_dict['dates']['start'], '%Y/%m/%d'),
           until=datetime.strptime(config.config_dict['dates']['end'], '%Y/%m/%d')):
    if dt not in vacation_dt and datetime.weekday(dt) in range(0, 5):
            days.append(dt)